# Solver debugging capabilities

When setting up a model or a use case, it's pretty often handy to be able to introspect the solver (e.g. plot intermediate resolution steps).

In [1]:
from cosapp.recorders import DataFrameRecorder
from cosapp.utils import set_log, LogLevel

from cpu.systems import CPUSystem
from cpu.utils.plot_solver_trace import plot_solver_trace

cpu = CPUSystem("cpu")
set_log(level=LogLevel.DEBUG)

In [2]:
from cosapp.drivers import NonLinearSolver, RunSingleCase

design = cpu.add_driver(NonLinearSolver("solver", factor=0.1))
runner = design.add_driver(RunSingleCase("runner"))
design.extend(cpu.design_methods["exchanger_surface"])
runner.set_values({"fan.T_air": 40.0, "T_cpu": 80.0, "cpu.usage": 100.0})

rec = runner.add_recorder(
    DataFrameRecorder(
        includes=[
            "T_cpu",
            "cpu.power",
            "cpu.usage",
            "cpu.heat_flow",
            "exchanger.heat_flow",
            "exchanger.h",
            "exchanger.surface",
            "fan.tension",
            "fan.fl_out.mass_flow",
            "controler.T",
            "controler.tension",
        ],
        hold=False,
    )
)
# run design
cpu.run_drivers()

System <cpu - CPUSystem> is the execution master.
Call cpu.open_loops
Call controler.open_loops
Call fan.open_loops
Call exchanger.open_loops
Call cpu.open_loops
Start setup_run recursive calls.
Call solver.setup_run
Call runner.setup_run
****************************************
*
* Assemble mathematical problem
*
****************************************
Mathematical problem:
Unknowns [1]
  exchanger.surface = 0.01
Equations [1]
  cpu.heat_flow == cpu.power := -20.0
Call cpu.setup_run
Call controler.setup_run
Call fan.setup_run
Call exchanger.setup_run
Call cpu.setup_run
Exec order for cpu: ['controler', 'fan', 'exchanger', 'cpu']
Call driver solver.run_once on cpu
 ------------------------------------------------------------
 # Starting driver 'solver' on 'cpu'
Call solver.compute_before()
Set unknowns initial values: [0.01]
Call solver.compute()
NR - Reference call
Call fresidues with x = array([0.01])
Call runner.run_once()
Entering runner (on System 'cpu') - RunSingleCase - run_onc

In [3]:
rec.data

,Section,Status,Error code,Reference,T_cpu,controler.tension,cpu.heat_flow,cpu.power,cpu.usage,exchanger.h,exchanger.heat_flow,exchanger.surface,fan.fl_out.mass_flow,fan.tension
0,,,0,runner,80.0,12.0,105.0,105.0,100.0,310.0,105.0,0.008468,1.0,12.0


In [4]:
plot_solver_trace(design)

In [5]:
from cosapp.systems import System


class ParabolicSystem(System):

    def setup(self):
        self.add_inward("a", 2.0)
        self.add_inward("x", 1.0)

        self.add_outward("y")

    def compute(self):
        self.y = self.a * self.x**2


p = ParabolicSystem("p")

p_design = p.add_driver(NonLinearSolver("solver", factor=1.0))
p_design.add_equation("y == 10.").add_unknown("x")
p.run_drivers()

plot_solver_trace(p_design)

System <p - ParabolicSystem> is the execution master.
Call p.open_loops
Start setup_run recursive calls.
Call solver.setup_run
****************************************
*
* Assemble mathematical problem
*
****************************************
Mathematical problem:
Unknowns [1]
  x = 1.0
Equations [1]
  y == 10. := -9.0
Call p.setup_run
Exec order for p: []
Call driver solver.run_once on p
 ------------------------------------------------------------
 # Starting driver 'solver' on 'p'
Call solver.compute_before()
Set unknowns initial values: [1.]
Call solver.compute()
NR - Reference call
Call fresidues with x = array([1.])
Call p.compute_before
Residues: array([-8.])
Initial residue: 8.0
Reuse of previous Jacobian matrix
Iteration 0
Perturb unknown 0
Call fresidues with x = array([1.00001526])
Call p.compute_before
Residues: array([-7.99993896])
Jacobian matrix: full update
Call fresidues with x = array([2.99998474])
Call p.compute_before
Residues: array([7.9998169])
Residue: 7.9998
i